In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Using TensorFlow backend.


In [12]:
non_best_sellers = pd.read_csv('non_best_sellers.csv')
best_sellers = pd.read_csv('best_sellers.csv')
best_sellers['image'] = best_sellers.pop('imagelist')
train = best_sellers.append(non_best_sellers)
train.pop('Unnamed: 0')

0       0
1       1
2       2
3       3
4       4
       ..
2616    0
2617    0
2618    0
2619    0
2620    0
Name: Unnamed: 0, Length: 4854, dtype: int64

In [29]:
train_image = []
best_list = []
for i in tqdm(range(len(train))):
    if list(train.best)[i] == 1:
        folder = 'best_sellers_images'
        best_list.append(1)
    else:
        folder = 'non_best_sellers_images'
        best_list.append(0)
    img = image.load_img(folder+'/'+list(train.image)[i], target_size=(28,28,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)
y = np.array(best_list)

100%|██████████| 4854/4854 [00:19<00:00, 244.48it/s]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)